In [12]:
import pandas as pd
import numpy as np
import pickle as pkl
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [13]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
BRAIN_STATUS = '1LNAdl2GTLYQfIM9mWFzn4cqAAjIWE5pjiJxkbEyQuQA'
RANGE_NAME = 'Sheet1!A1:K'

def get_brain_status_spreadsheet():
    """
    This is a basic utility function in order to scrape the 
    status brain google sheet so that we don't have to 
    manually fill this in. It will return a pandas dataframe. 
    
    """
    creds = None
    # The file token.pkl stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pkl'):
        with open('token.pkl', 'rb') as token:
            creds = pkl.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.pkl', 'wb') as token:
            pkl.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=BRAIN_STATUS,
                                range=RANGE_NAME).execute()
    values = result.get('values', [])
    df = pd.DataFrame(values, index = None)
    return df


In [14]:
get_brain_status_spreadsheet()

,0,1,2,3,4,5,6,7,8,9,10
0,Mouse,DOB,Surgery date,Age group,Sex,Quantification status,Notes,Issues,CZI file name,Count CSV File Name,Analysis Status
1,CRWT58DNMF,8/18/18,10/10/18,Adult,F,None,None,None,None,None,None
2,CRANX33URTF,9/13/18,10/4/18,Juvenile,F,,,,2018_07_20_RecognizeCode,None,None
3,CRANX33JNM,5/29/18,,Juvenile,M,None,None,None,None,None,None
4,CRWT57CNM,8/16/18,10/8/18,Adult,M,None,None,None,None,None,None
5,NJO069 (CRANX34ART),3/28/18,5/19/18,Adult,M,None,None,None,None,None,None
6,NJO013,10/22/17,12/5/17,Adult,M,None,None,None,None,None,None
